In [9]:
import re
import itertools
from dataclasses import dataclass
from more_itertools import chunked

In [10]:
@dataclass
class Pattern:
    name: str
    pattern: bytes

In [11]:
def hex_to_bytes(s: str) -> bytes:
    return int(s, 16).to_bytes(1, "little")

In [12]:
def convert_to_pattern(s: list[str]):
    pattern = b"".join(b"." if item == "??" else re.escape(hex_to_bytes(item)) for item in s)
    return pattern

In [13]:
patterns = []

with open("fn_byte_patterns.ffsess") as patterns_file:
    for tab_line, pattern_line in chunked(patterns_file, 2):
        command, _, tab_name = tab_line.rstrip().partition(" ")
        assert command == "Tab", command
        rule_name, _, _, *pattern = pattern_line.rstrip().split(" ")
        assert rule_name == "RuleBytePattern"
        patterns.append(Pattern(tab_name, convert_to_pattern(pattern)))

In [14]:
patterns

[Pattern(name='string_copy', pattern=b'H\x89Q.H\x83y..r.H\x8b\x01\xc6\x04\x10.\xc3\xc6\x04\x11.\xc3'),
 Pattern(name='string_copy_n', pattern=b'H\x89\\\\\\$.H\x89l\\$.VWAWH\x83\xec.H\x8bi.I\x8b\xf0L\x8b\xfaH\x8b\xd9L;\xc5w.H\x8b\xf9H\x83\xfd.r.H\x8b9H\x89q.H\x8b\xcf\xe8....\xc6\x047.\xe9....H\xbf........H;\xf7\x0f\x87....H\x8b\xceL\x89t\\$.H\x83\xc9.H;\xcfw.H\x8b\xd5H\x8b\xc7H\xd1\xeaH\\+\xc2H;\xe8w.H\x8d\x04\\*H\x8b\xf9H;\xc8H\x0fB\xf8H\x8dO.\xe8....L\x8b\xc6H\x89s.I\x8b\xd7H\x89\\{.H\x8b\xc8L\x8b\xf0\xe8....A\xc6\x046.H\x83\xfd.r.H\x8b\\\x0bH\x8dU.H\x81\xfa....r.L\x8bA.H\x83\xc2.I\\+\xc8H\x8dA.H\x83\xf8.w.I\x8b\xc8\xe8....L\x893L\x8bt\\$.H\x8bl\\$.H\x8b\xc3H\x8b\\\\\\$.H\x83\xc4.A__\\^\xc3\xff\x15....\xcc\xe8....\xcc'),
 Pattern(name='string_append', pattern=b'I\xc7\xc0....f\x0f\x1f\x84\x00....I\xff\xc0B\x80<\x02.u.\xe9....'),
 Pattern(name='string_append_0', pattern=b'I\xc7\xc0....f\x0f\x1f\x84\x00....I\xff\xc0B\x80<\x02.u.\xe9....'),
 Pattern(name='string_append_n', pattern=b'H\x89

In [15]:
path = "/mnt/second/SteamLibrary/steamapps/common/Dwarf Fortress/Dwarf Fortress.exe"

In [16]:
with open(path, "rb") as file:
    data = file.read()
    for pattern in patterns:
        found = False
        for match in re.finditer(pattern.pattern, data):
            print(pattern.name, hex(match.start(0)))
            found = True
        if not found:
            print(pattern.name, "not found")

string_copy 0xac30
string_copy_n 0xac50
string_append 0xa9f0
string_append 0xaa10
string_append_0 0xa9f0
string_append_0 0xaa10
string_append_n 0xad90
convert_ulong_to_string 0x392c10
addst 0x7c4620
addst_top 0x7c4700
addcoloredst 0x7c42d0
addst_flag 0x7c43b0
standartstringentry 0x8c3380
simplify_string 0x393750
upper_case_string 0x393a90
lower_case_string 0x3938f0
capitalize_string_words 0x393c30
capitalize_string_first_word 0x393eb0
addchar 0x555e0
addchar 0xf3110
addchar_top 0x555e0
addchar_top 0xf3110
addtexture 0xedaf20
gps_allocate not found
cleanup_arrays not found
screen_to_texid 0x5f6850
screen_to_texid_top 0x5f6a40
loading_world_new_game_loop not found
loading_world_continuing_game_loop 0x5a0c10
loading_world_start_new_game_loop 0x59e860
menu_interface_loop not found
